<a href="https://colab.research.google.com/github/yzheng2001/AI/blob/main/CNN_digtImage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

convolution nn: not fullly connected as regular nn, as image has a lot of data, fully connected nn will take forever to train
//color images are 3-d data (height, width, red), (height, width, green), (height, width, blue)
pooling  = down sampling (max pooling, avg pooling)
kernel means filter matrix

image set for training: https://www.tensorflow.org/datasets/catalog/mnist

conceptual steps of cnn
feature extraction step: contain several rounds of convolution (to get feature maps) and polling (to get pool fetured maps), eventually gets a falttern layer
classification steps: using fully connected cnn to eventually get the probablity tensor of the output feature
https://images.app.goo.gl/rM5FA5nfHZxBXywv5\

A phython indentation tip:Each block must be indented by one level relative to the previous block, but the amount of indentation in a level is not specified by the language, and can vary from one block to another. So long each block is consistent, the interpreter is satisfied. You can choose 2, 4, 8 spaces or a tab, just be consistent

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline
#
# plot in colab

In [2]:
# test data: MNIST is black and white number images 28x28,

transform = transforms.ToTensor()
#train_data = datasets.MNIST() #there are other than MNIST dataset in datasets
#get training data save to local colab /cnn_data, use above transformer to cnvert MNIST image files into a tensor of 4-d(# of images, height, width, color channel)
train_data = datasets.MNIST(root='cnn_data', train=True, download=True, transform=transform)  #save to local colab /cnn_data

# test data
test_data = datasets.MNIST(root='cnn_data', train=False, download=True, transform=transform)  #already saved to local colab /cnn_data above so here no more download, and also train = False to get test data in the MNIST


In [3]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [4]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [5]:
#pwd
#ls

NameError: name 'pwd' is not defined

In [6]:
#batch size better be small for images, 3~5, but here 10 images per batch
train_loader= DataLoader(train_data, batch_size=10, shuffle= True)
test_loader= DataLoader(test_data, batch_size=10, shuffle= False)

In [7]:
# define cnn Model
conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=3, stride=1) #same as conv1 = nn.Conv2d(1, 6, 3, 1)
conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=3, stride=1)


In [8]:
#grab 1 MNIST record/image
for i, (X_Train, y_train) in enumerate(train_data):
  break

In [9]:
X_Train.shape #torch.Size([1, 28, 28]) , where 1 means 1 image

torch.Size([1, 28, 28])

In [10]:
#reshape (or view) the tensor
x= X_Train.view(1,1,28,28) #1 batch of 1 image of size 28x28

In [11]:
# Perfrm our 1st convlution, # Rectified Linear Unit for our activation function
x=F.relu(conv1(x))

In [12]:
#1 single image, 6 filters in the first layer, 26x26 is the output because we didn't set padding in the nn.Conv2d above, as MNIST images has not much iformation around the edge
x.shape

torch.Size([1, 6, 26, 26])

In [13]:
# pass thru the pooling layer, with kernal size of 2, stride of 2
x =F.max_pool2d(x, 2, 2)
x.shape # 26/3=13, max pooling to shrink down the first convolutional layer result

torch.Size([1, 6, 13, 13])

In [15]:
x=F.max_pool2d(x, 2,2)
x.shape # 11/2 = 5 after round down, meanning the image are size of 5x5, and this layer has 16 output feature

torch.Size([1, 16, 5, 5])

In [14]:
# Do our second  convolutional layer, no padding so loosing 2 pixels around the outside fo the image
x = F.relu(conv2(x))
x.shape

torch.Size([1, 16, 11, 11])

In [17]:
# Above was playingn with 1 image, now define the CNN module, adding above Conv2d (cnn) and the classification/traditional nn (nn.Linear, fully connected)
# Model Class
class ConvolutionalNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1,6,3,1) #Conv2d takes in 2d image/data, no need to flatern the image; 1 image in, 6 nodes output, kernel size 3 and stride 1
    self.conv2 = nn.Conv2d(6,16,3,1)
    # Fully Connected Layer, input data size is 5*5*16 (16 output channel, and image size 5x5),  10 is decided because output_feature are 0~9 digits from the image
    # but 120 and 84 are arbitratry, just need to get smaller each layer
    self.fc1 = nn.Linear(5*5*16, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, X):
    X = F.relu(self.conv1(X))
    X = F.max_pool2d(X,2,2) # 2x2 kernal and stride 2
    # Second Pass
    X = F.relu(self.conv2(X))
    X = F.max_pool2d(X,2,2) # 2x2 kernal and stride 2

    # reshape/Re-View to flatten it out
    X = X.view(-1, 16*5*5) # negative one so that we can vary the batch size

    # Fully Connected Layers
    X = F.relu(self.fc1(X))
    X = F.relu(self.fc2(X))
    X = self.fc3(X)
    return F.log_softmax(X, dim=1) #Apply a softmax followed by a logarithm??? # supposedly return is a digit in 0~9 ???

In [18]:
# Create an Instance of our Model
torch.manual_seed(41)
model = ConvolutionalNetwork()
model

ConvolutionalNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [21]:
# not sure what is model.parameters(), but just know it is usually used as input for optimizer
for param in model.parameters():
  print(type(param), param.size())
# <class 'torch.Tensor'> (20L,)
# <class 'torch.Tensor'> (20L, 1L, 5L, 5L)

# Loss Function Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # Smaller the Learning Rate, longer its gonna take to train.

<class 'torch.nn.parameter.Parameter'> torch.Size([6, 1, 3, 3])
<class 'torch.nn.parameter.Parameter'> torch.Size([6])
<class 'torch.nn.parameter.Parameter'> torch.Size([16, 6, 3, 3])
<class 'torch.nn.parameter.Parameter'> torch.Size([16])
<class 'torch.nn.parameter.Parameter'> torch.Size([120, 400])
<class 'torch.nn.parameter.Parameter'> torch.Size([120])
<class 'torch.nn.parameter.Parameter'> torch.Size([84, 120])
<class 'torch.nn.parameter.Parameter'> torch.Size([84])
<class 'torch.nn.parameter.Parameter'> torch.Size([10, 84])
<class 'torch.nn.parameter.Parameter'> torch.Size([10])


In [22]:
import time
start_time = time.time()

# Create Variables To Tracks Things
epochs = 5
train_losses = []
test_losses = []
train_correct = []
test_correct = []

# For Loop of Epochs
for i in range(epochs):
  trn_corr = 0
  tst_corr = 0


  # Train
  for b,(X_train, y_train) in enumerate(train_loader):
    b+=1 # start our batches at 1
    #why we need to manually do b+=1, python should auto-add-1 here as in c#

    y_pred = model(X_train) # get predicted values from the training set. Not flattened 2D
    loss = criterion(y_pred, y_train) # how off are we? Compare the predictions to correct answers in y_train

    predicted = torch.max(y_pred.data, 1)[1] # add up the number of correct predictions. Indexed off the first point ???
    batch_corr = (predicted == y_train).sum() # how many we got correct from this batch. True = 1, False=0, sum those up
    trn_corr += batch_corr # keep track as we go along in training.

    # Update our parameters
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


    # Print out some results every 600 batches
    if b%600 == 0:
      print(f'Epoch: {i}  Batch: {b}  Loss: {loss.item()}')

  train_losses.append(loss)
  train_correct.append(trn_corr)

  # Test
  with torch.no_grad(): #No gradient so we don't update our weights and biases with test data
    for b,(X_test, y_test) in enumerate(test_loader):
      y_val = model(X_test)
      predicted = torch.max(y_val.data, 1)[1] # Adding up correct predictions
      tst_corr += (predicted == y_test).sum() # T=1 F=0 and sum away


  loss = criterion(y_val, y_test)
  test_losses.append(loss)
  test_correct.append(tst_corr)



current_time = time.time()
total = current_time - start_time
print(f'Training Took: {total/60} minutes!')

Epoch: 0  Batch: 600  Loss: 0.1623610556125641
Epoch: 0  Batch: 1200  Loss: 0.1502392590045929
Epoch: 0  Batch: 1800  Loss: 0.4744560718536377
Epoch: 0  Batch: 2400  Loss: 0.14238706231117249
Epoch: 0  Batch: 3000  Loss: 0.007758188061416149
Epoch: 0  Batch: 3600  Loss: 0.3836284875869751
Epoch: 0  Batch: 4200  Loss: 0.0038223876617848873
Epoch: 0  Batch: 4800  Loss: 0.0021286322735249996
Epoch: 0  Batch: 5400  Loss: 0.0569545142352581
Epoch: 0  Batch: 6000  Loss: 0.00038789428072050214
Epoch: 1  Batch: 600  Loss: 0.005851339548826218
Epoch: 1  Batch: 1200  Loss: 0.3855525553226471
Epoch: 1  Batch: 1800  Loss: 0.004819948226213455
Epoch: 1  Batch: 2400  Loss: 0.003216963727027178
Epoch: 1  Batch: 3000  Loss: 0.0332382395863533
Epoch: 1  Batch: 3600  Loss: 0.5372857451438904
Epoch: 1  Batch: 4200  Loss: 0.04561494290828705
Epoch: 1  Batch: 4800  Loss: 0.0007510822033509612
Epoch: 1  Batch: 5400  Loss: 0.0001173773780465126
Epoch: 1  Batch: 6000  Loss: 0.14201366901397705
Epoch: 2  Batch